In [1]:
import os
import argparse
from argparse import Namespace
import pathlib
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import esm
from esm.data import *
from esm.model.esm2_secondarystructure import ESM2 as ESM2_SISS
from esm.model.esm2_supervised import ESM2
from esm import Alphabet, FastaBatchedDataset, ProteinBertModel, pretrained, MSATransformer
from esm.modules import ConvTransformerLayer

import numpy as np
import pandas as pd
import random
import math
import scipy.stats as stats
from scipy.stats import spearmanr, pearsonr
from sklearn.metrics import r2_score, f1_score, roc_auc_score, mean_squared_error, mean_absolute_error
from sklearn import preprocessing
from copy import deepcopy
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from sklearn.utils import class_weight
from torch.utils.data import RandomSampler, SequentialSampler

from collections import Counter
os.chdir('/devdata/pansc/github/')

global layers, heads, embed_dim, batch_toks, cnn_layers, alphabet


In [2]:
layers = 6
heads = 16
embed_dim = 128
# batch_toks = 4096*2 #4096
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')
repr_layers = [0, layers]

In [3]:
class ConvTransformerPredictor(nn.Module):
    def __init__(self, alphabet, dropout=0.2, CovTransformer_layers=3, 
                 kmer=7, layers=6, embed_dim=128, nodes=40, heads=16):
        super(ConvTransformerPredictor, self).__init__()
        self.embedding_size = embed_dim
        self.nodes = nodes
        self.dropout = dropout
        self.esm2 = ESM2_SISS(num_layers = layers,
                        embed_dim = embed_dim,
                        attention_heads = heads,
                        alphabet = alphabet) 
        # 修改为 nn.ModuleList
        self.convtransformer_decoder = nn.ModuleList([
            ConvTransformerLayer(embed_dim, embed_dim*4, heads, kmer-i*2, dropout=self.dropout, use_esm1b_layer_norm=True) #(kmer-i*2)
            for i in range(CovTransformer_layers)
        ])
        self.dropout = nn.Dropout(self.dropout)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        # 处理实验来源信息的线性层
        self.experiment_dense = nn.Linear(2, self.nodes)  # 处理 one-hot 实验指示符
        self.linear = nn.Linear(in_features = 6 * embed_dim, out_features = self.nodes)
        self.linear_2 = nn.Linear(in_features = self.nodes, out_features = self.nodes * 4)
        self.linear_3 = nn.Linear(in_features = self.nodes * 4, out_features = self.nodes)
        self.output = nn.Linear(in_features = self.nodes, out_features = 1)

    def forward(self, tokens, experiment_indicator, self_attn_padding_mask=None):
        # ESM embedding
        embeddings = self.esm2(tokens, repr_layers, return_representation=True)
        embeddings_rep = embeddings["representations"][layers][:, 1 : -1] #B*(T+2)*E -> B*T*E

        for i, layer in enumerate(self.convtransformer_decoder):
            x_o, attn = layer(x=embeddings_rep, self_attn_padding_mask=self_attn_padding_mask)  #tokens: B*T*E, x_o: B*T*E

        x = torch.flip(x_o, dims=[1])  # Reverse along the sequence length dimension
        # Select frames corresponding to frame 1, frame 2, and frame 3
        frame_1 = x[:, 0::3, :]
        frame_2 = x[:, 1::3, :]
        frame_3 = x[:, 2::3, :]
        # 全局最大池化
        frame_1_max = torch.max(frame_1, dim=1)[0]  # B*C
        frame_2_max = torch.max(frame_2, dim=1)[0]  # B*C
        frame_3_max = torch.max(frame_3, dim=1)[0]  # B*C
        # 扩展 self_attn_padding_mask 的维度以匹配特征张量
        mask_expanded = ~self_attn_padding_mask.unsqueeze(2)  # (batch_size, seq_len, 1)，True 表示有效数据
        # 计算有效位置的均值池化
        def masked_mean(frame, mask):
            frame_sum = torch.sum(frame * mask, dim=1)
            mask_sum = torch.sum(mask, dim=1) + 1e-8  # 避免除零
            return frame_sum / mask_sum
        # 全局均值池化
        frame_1_avg = masked_mean(frame_1, mask_expanded[:, 0::3, :])
        frame_2_avg = masked_mean(frame_2, mask_expanded[:, 1::3, :])
        frame_3_avg = masked_mean(frame_3, mask_expanded[:, 2::3, :])
        # 将池化后的张量拼接为一个张量
        pooled_output = torch.cat([frame_1_max, frame_1_avg, frame_2_max, frame_2_avg, frame_3_max, frame_3_avg], dim=1)  # B*(6*C)
        # 线性层处理实验指示符
        experiment_output = self.experiment_dense(experiment_indicator)
        x_pooled = self.flatten(pooled_output)

        o_linear = self.linear(x_pooled) + experiment_output #将池化输出与实验信息拼接
        o_linear_2 = self.linear_2(o_linear)
        o_linear_3 = self.linear_3(o_linear_2)

        o_relu = self.relu(o_linear_3)
        o_dropout = self.dropout(o_relu)
        o = self.output(o_dropout)  # B*1

        return o

In [4]:
def r2(x,y):
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    return r_value**2

def performances(label, pred):
    label, pred = list(label), list(pred)
    r = r2(label, pred)
    R2 = r2_score(label, pred)
    rmse = np.sqrt(mean_squared_error(label, pred))
    mae = mean_absolute_error(label, pred)
    try:
        pearson_r = pearsonr(label, pred)[0]
    except:
        pearson_r = -1e-9
    try:
        sp_cor = spearmanr(label, pred)[0]
    except:
        sp_cor = -1e-9
    print(f'r-squared = {r:.4f} | pearson r = {pearson_r:.4f} | spearman R = {sp_cor:.4f} | R-squared = {R2:.4f} | RMSE = {rmse:.4f} | MAE = {mae:.4f}')
    return [r, pearson_r, sp_cor, R2, rmse, mae]

def performances_to_pd(performances_list):
    performances_pd = pd.DataFrame(performances_list, index = ['r2', 'PearsonR', 'SpearmanR', 'R2', 'RMSE', 'MAE']).T
    return performances_pd

In [5]:
def generate_dataset_dataloader(e_data, obj_col, lab_col, batch_toks=8192, mask_prob = 0.0):
    dataset = FastaBatchedDataset(e_data.loc[:,obj_col], e_data.loc[:, lab_col], mask_prob = mask_prob)
    batches = dataset.get_batch_indices(toks_per_batch=batch_toks, extra_toks_per_seq=2)
    dataloader = torch.utils.data.DataLoader(dataset, 
                                            collate_fn=alphabet.get_batch_converter(), 
                                            batch_sampler=batches, 
                                            shuffle = False)
    print(f"{len(dataset)} sequences")
    return dataset, dataloader, batches

def get_experiment_indicator_for_batch(data_combine, batch_idx):
    # 从 train_combine 中获取对应 batch 的 experiment_indicator
    batch_experiment_indicators = data_combine.iloc[batch_idx]['experiment_indicator'].values.tolist()
    # 转换为 tensor
    experiment_indicator_tensor = torch.tensor(batch_experiment_indicators, dtype=torch.float32).to(device)
    return experiment_indicator_tensor

def shuffle_data_fn(in_data):
    # 使用 sample(frac=1) 来打乱数据集顺序
    shuffle_data = in_data.sample(frac=1).reset_index(drop=True)
    return shuffle_data

In [6]:
def train_step(train_dataloader, train_shuffle_combine, train_shuffle_batch, model, epoch):        
    model.train()
    y_pred_list, y_true_list, loss_list = [], [], []
    
    for i, (labels, strs, masked_strs, toks, masked_toks, _) in enumerate(tqdm(train_dataloader)):
        toks = toks.to(device)
        padding_mask = toks.eq(alphabet.padding_idx)[:, 1:-1]
        labels = torch.FloatTensor(labels).to(device).reshape(-1, 1)
        experiment_indicator_tensor = get_experiment_indicator_for_batch(train_shuffle_combine, train_shuffle_batch[i])

        outputs= model(toks, experiment_indicator_tensor, self_attn_padding_mask=padding_mask)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_list.append(loss.cpu().detach())

        y_true_list.extend(labels.cpu().reshape(-1).tolist())
        y_pred = outputs.reshape(-1).cpu().detach().tolist()
        y_pred_list.extend(y_pred)
        
    loss_epoch = float(torch.Tensor(loss_list).mean())
    print(f'Train: Epoch-{epoch}/{num_epochs} | Loss = {loss_epoch:.4f} | ', end = '')
    
    metrics = performances(y_true_list, y_pred_list)
    return metrics, loss_epoch

def eval_step(test_dataloader, test_combine, test_batch, model, epoch):
    model.eval()
    y_pred_list, y_true_list, loss_list = [], [], []
    strs_list = []
    with torch.no_grad():
        for i, (labels, strs, masked_strs, toks, masked_toks, _) in enumerate(tqdm(test_dataloader)):
            strs_list.extend(strs)
            toks = toks.to(device)
            padding_mask = toks.eq(alphabet.padding_idx)[:, 1:-1]
            labels = torch.FloatTensor(labels).to(device).reshape(-1, 1)
            experiment_indicator_tensor = get_experiment_indicator_for_batch(test_combine, test_batch[i])
            
            outputs= model(toks, experiment_indicator_tensor, self_attn_padding_mask=padding_mask)
            loss = criterion(outputs, labels)
            loss_list.append(loss.cpu().detach())

            y_pred = outputs.reshape(-1).cpu().detach().tolist()
            y_true_list.extend(labels.cpu().reshape(-1).tolist())
            y_pred_list.extend(y_pred)
        
        loss_epoch = float(torch.Tensor(loss_list).mean())
        print(f'Test: Epoch-{epoch}/{num_epochs} | Loss = {loss_epoch:.4f} | ', end = '')
        metrics = performances(y_true_list, y_pred_list)
        e_pred = pd.DataFrame([strs_list, y_true_list, y_pred_list], index = ['utr', 'y_true', 'y_pred']).T
        
    return metrics, loss_epoch, e_pred


In [7]:
cnn_pred_mrl_random_50 = pd.read_csv('/devdata/pansc/github/UTR_Insight/Result/optimus_framepool/4.1_test_data_GSM3130435_egfp_unmod_1_pred_mrl.csv')
cnn_pred_mrl_human_100 = pd.read_csv('/devdata/pansc/github/UTR_Insight/Result/optimus_framepool/VaryLengthHumanTest_sequence_num7600_pred_mrl.csv')
cnn_pred_mrl_random_100 = pd.read_csv('/devdata/pansc/github/UTR_Insight/Result/optimus_framepool/VaryLengthRandomTest_sequence_num7600_pred_mrl.csv')

lm_pred_mrl_random_50 = pd.read_csv('/devdata/pansc/github/UTR_Insight/Result/utr_insight/e_pred_random_50.csv')
lm_pred_mrl_human_100 = pd.read_csv('/devdata/pansc/github/UTR_Insight/Result/utr_insight/e_pred_human_100.csv')
lm_pred_mrl_random_100 = pd.read_csv('/devdata/pansc/github/UTR_Insight/Result/utr_insight/e_pred_random_100.csv')

In [8]:
lm_pred_mrl_human_100

In [35]:
cnn_results = [cnn_pred_mrl_random_50, cnn_pred_mrl_random_100, cnn_pred_mrl_human_100]
lm_results = [lm_pred_mrl_random_50, lm_pred_mrl_random_100, lm_pred_mrl_human_100]

var_prefix = ['random_50', 'random_100', 'human_100']
var_prefix_vary = ['random_100', 'human_100']

In [10]:
# 初始化空列表存储每次循环生成的DataFrame
all_results = []

for i in range(len(cnn_results)):
    rl = cnn_results[i].loc[:, 'rl']
    rl_lm = lm_results[i].loc[:, 'y_true']
    
    optimus_pred = cnn_results[i].loc[:, 'pred_optimus']
    framepool_pred = cnn_results[i].loc[:, 'pred_frame_pool']
    lm_pred = lm_results[i].loc[:, 'y_pred']

    optimus_per = performances(rl, optimus_pred)
    framepool_per = performances(rl, framepool_pred)
    lm_per = performances(rl_lm, lm_pred)

    optimus_pd = performances_to_pd(optimus_per)
    framepool_pd = performances_to_pd(framepool_per)
    lm_pd= performances_to_pd(lm_per)

    # 为每个DataFrame添加“数据集”和“模型”列
    optimus_pd['Dataset'] = var_prefix[i]
    optimus_pd['Model'] = 'optimus'
    
    framepool_pd['Dataset'] = var_prefix[i]
    framepool_pd['Model'] = 'framepool'

    lm_pd['Dataset'] = var_prefix[i]
    lm_pd['Model'] = 'utr_insignt'
    
    # 将每个DataFrame存储到all_results列表中
    all_results.append(optimus_pd)
    all_results.append(framepool_pd)
    all_results.append(lm_pd)

# 将所有的DataFrame拼接成一个大DataFrame
final_df = pd.concat(all_results, ignore_index=True)


In [11]:
# 获取当前的列名列表
cols = final_df.columns.tolist()
new_order = ['Model', 'Dataset'] + [col for col in cols if col not in ['Dataset', 'Model']]
final_df = final_df[new_order]
# 打印结果以检查
# final_df.to_csv('/devdata/pansc/github/UTR-LM/Scripts/UTRLM_downstream/model_compare.csv')


In [12]:
utr_lm_pred_mrl_random_50 = pd.read_csv('/devdata/pansc/github/UTR_Insight/Result/utr_lm/UTR_LM_prediction_random_50.csv')

In [13]:
utr_lm_rl = utr_lm_pred_mrl_random_50.loc[:, 'rl']
utr_lm_pred = utr_lm_pred_mrl_random_50.loc[:, 'MRL']
utr_lm_per = performances(utr_lm_rl, utr_lm_pred)
utr_lm_pd= performances_to_pd(utr_lm_per)

In [20]:
cnn_results_new = []

# 遍历cnn_results数组并更新每个DataFrame
for i in range(len(cnn_results)):
    # 提取需要的列
    cnn_results[i] = cnn_results[i][['id', 'utr', 'rl', 'pred_optimus', 'pred_frame_pool']]
    # 根据utr列计算序列长度，并添加新列'len'
    cnn_results[i]['len'] = cnn_results[i]['utr'].apply(len)
    cnn_results_new.append(cnn_results[i])

# 初始化一个新的列表存储处理后的DataFrame
lm_results_new = []

# 遍历lm_results数组并更新每个DataFrame
for df in lm_results:
    # 复制DataFrame，避免直接修改原数据
    df_new = df.copy() 
    # 去除utr列中的'<pad>'并存储为新的列raw_utr
    df_new['raw_utr'] = df_new['utr'].str.replace('<pad>', '', regex=False)
    df_new['len'] = df_new['raw_utr'].apply(len)
    # 将处理后的DataFrame存储到lm_results_new列表中
    lm_results_new.append(df_new)

In [26]:
cnn_results_vary = cnn_results_new[1:]
lm_results_vary = lm_results_new[1:]

In [36]:
# 定义序列长度的分段区间
length_bins = [(25, 44), (45, 64), (65, 84), (85, 100)]
length_labels = ['25-44', '45-64', '65-84', '85-100']

all_results_grpup = []
for i, (min_len, max_len) in enumerate(length_bins):
    for j in range(len(cnn_results_vary)):

        cnn_results_vary_group = cnn_results_vary[j][(cnn_results_vary[j]['len'] >= min_len) & (cnn_results_vary[j]['len'] <= max_len)]
        lm_results_vary_group = lm_results_vary[j][(lm_results_vary[j]['len'] >= min_len) & (lm_results_vary[j]['len'] <= max_len)]

        rl = cnn_results_vary_group.loc[:, 'rl']
        rl_lm = lm_results_vary_group.loc[:, 'y_true']

        optimus_pred = cnn_results_vary_group.loc[:, 'pred_optimus']
        framepool_pred = cnn_results_vary_group.loc[:, 'pred_frame_pool']
        lm_pred = lm_results_vary_group.loc[:, 'y_pred']

        optimus_per = performances(rl, optimus_pred)
        framepool_per = performances(rl, framepool_pred)
        lm_per = performances(rl_lm, lm_pred)

        optimus_pd = performances_to_pd(optimus_per)
        framepool_pd = performances_to_pd(framepool_per)
        lm_pd= performances_to_pd(lm_per)

        # 为每个DataFrame添加“数据集”和“模型”列和“长度分段”列
        optimus_pd['Dataset'] = var_prefix_vary[j]
        optimus_pd['Model'] = 'optimus'
        optimus_pd['group'] = length_labels[i]
        
        framepool_pd['Dataset'] = var_prefix_vary[j]
        framepool_pd['Model'] = 'framepool'
        framepool_pd['group'] = length_labels[i]

        lm_pd['Dataset'] = var_prefix_vary[j]
        lm_pd['Model'] = 'utr_insignt'
        lm_pd['group'] = length_labels[i]

        all_results_grpup.append(optimus_pd)
        all_results_grpup.append(framepool_pd)
        all_results_grpup.append(lm_pd)

# 将所有的DataFrame拼接成一个大DataFrame
final_df_group = pd.concat(all_results_grpup, ignore_index=True)


In [38]:
final_df_group.to_csv('/devdata/pansc/github/UTR_Insight/Result/model_compare/model_compare_group.csv')